In [1]:
import pandas as pd
from shapely.geometry import  Point
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing as prepro
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re
from matplotlib import pyplot as plt



In [7]:
pdir_data = pd.read_csv("resources/Police_Department_Incident_Reports__2018_to_Present (1)\Police_Department_Incident_Reports__2018_to_Present (1).csv")


In [8]:
# look at a few rows of the pdir_data file.
pdir_data.sample(5)

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts
396069,2018/11/24 10:19:00 AM,2018/11/24,10:19,2018,Saturday,2018/11/24 10:19:00 AM,74251304134,742513,180888495,183281035.0,...,-122.406527,POINT (-122.40652694268661 37.79694387820634),104.0,NaN,NaN,NaN,NaN,NaN,3.0,6.0
6981,2023/03/31 01:35:00 PM,2023/03/31,13:35,2023,Friday,2023/03/31 06:00:00 PM,126148506244,1261485,236058429,NaN,...,-122.412650,POINT (-122.41264965842102 37.79255233583978),16.0,NaN,NaN,NaN,NaN,NaN,3.0,6.0
708357,2020/11/16 09:00:00 AM,2020/11/16,09:00,2020,Monday,2020/11/25 05:06:00 PM,98161905073,981619,200713184,203302362.0,...,-122.461023,POINT (-122.46102312487574 37.76520933540097),109.0,NaN,NaN,NaN,NaN,NaN,11.0,7.0
263953,2022/12/10 07:30:00 PM,2022/12/10,19:30,2022,Saturday,2022/12/21 07:01:00 PM,122800809320,1228008,226249202,NaN,...,-122.419825,POINT (-122.41982523539403 37.76667360056878),53.0,NaN,NaN,NaN,3.0,NaN,2.0,3.0
375551,2018/06/08 06:15:00 PM,2018/06/08,18:15,2018,Friday,2018/06/09 09:01:00 AM,67770928150,677709,186128914,NaN,...,-122.424295,POINT (-122.42429481397211 37.78339985751909),100.0,NaN,NaN,NaN,NaN,NaN,11.0,4.0


In [9]:
# get the number of missing data points per column
missing_values_count = pdir_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Incident Datetime            0
Incident Date                0
Incident Time                0
Incident Year                0
Incident Day of Week         0
Report Datetime              0
Row ID                       0
Incident ID                  0
Incident Number              0
CAD Number              168715
dtype: int64

In [10]:
# how many total missing values do we have?
total_cells = np.product(pdir_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

17.619319660017755

In [11]:
# remove all the rows that contain a missing value
pdir_data.dropna()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts


In [12]:
# remove all columns with at least one missing value
columns_with_na_dropped = pdir_data.dropna(axis=1)
columns_with_na_dropped.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,Report Type Code,Report Type Description,Incident Code,Incident Description,Resolution,Police District
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,II,Coplogic Initial,6374,"Theft, Other Property, >$950",Open or Active,Mission
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,VS,Vehicle Supplement,7041,"Vehicle, Recovered, Auto",Open or Active,Out of SF


In [21]:
new_column_names = ['Incident_Datetime', 'Incident_Date', 'Incident_Time', 'Incident_Year', 'Incident_Day_of_Week',	'Report_Datetime', 'Row_ID',	'Incident_ID',	'Incident_Number',	'Report_Type_Code',	'Report_Type_Description',	'Incident_Code',	'Incident_Description',	'Resolution',	'Police_District' ]  # Provide a list of new column names
columns_with_na_dropped.columns = new_column_names
columns_with_na_dropped.head()

,Incident_Datetime,Incident_Date,Incident_Time,Incident_Year,Incident_Day_of_Week,Report_Datetime,Row_ID,Incident_ID,Incident_Number,Report_Type_Code,Report_Type_Description,Incident_Code,Incident_Description,Resolution,Police_District
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,VS,Vehicle Supplement,7041,Vehicle Recovered Auto,Open or Active,Out of SF
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,II,Coplogic Initial,6374,Theft Other Property >$950,Open or Active,Mission
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,VS,Vehicle Supplement,7041,Vehicle Recovered Auto,Open or Active,Out of SF
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,VS,Vehicle Supplement,7041,Vehicle Recovered Auto,Open or Active,Out of SF
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,VS,Vehicle Supplement,7041,Vehicle Recovered Auto,Open or Active,Out of SF


In [22]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % pdir_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 35 

Columns with na's dropped: 15


In [23]:
pdir_data.duplicated().sum()

0

In [24]:
columns_with_na_dropped = columns_with_na_dropped.astype(str)
columns_with_na_dropped = columns_with_na_dropped.applymap(lambda x: x.replace(',', ''))


In [27]:
columns_with_na_dropped.to_csv("resources/cleaned.csv", index=False)